In [45]:
from symbolic_bounds import ProgramFactory
from symbolic_bounds.dag import DAG

# Test with simple DAG: X -> Y
print("="*80)
print("SIMPLE EXAMPLE: Z -> X -> Y")
print("="*80)

# Create a DAG
dag = DAG()

# Add binary nodes - all nodes must be assigned to a partition
# Z = dag.add_node('Z', support={0, 1}, partition='L')
X = dag.add_node('X', support={0, 1}, partition='L')
Y = dag.add_node('Y', support={0, 1}, partition='R')
# dag.add_edge(Z, Y)
dag.add_edge(X, Y)

dag.generate_all_response_types()
# dag.print_response_type_table(Z)
dag.print_response_type_table(X)
dag.print_response_type_table(Y)

constraints1 = ProgramFactory.write_constraints(dag)
constraints1.print_constraints(show_matrices=False, explicit_equations=True)

# print("\n\n")

# Test with complex DAG: Z -> X -> Y, Z -> Y
# print("="*80)
# print("COMPLEX EXAMPLE: Z -> X -> Y, Z -> Y")
# print("="*80)

# constraints2 = ProgramFactory.write_constraints(dag2)
# constraints2.print_constraints(show_matrices=False)

SIMPLE EXAMPLE: Z -> X -> Y

Response types for X (no parents):
r_X^1: X = 0
r_X^2: X = 1


Response types for Y (parents: X):
=====

r_Y^1:
X | Y
-----
0 | 0
1 | 0

r_Y^2:
X | Y
-----
0 | 0
1 | 1

r_Y^3:
X | Y
-----
0 | 1
1 | 0

r_Y^4:
X | Y
-----
0 | 1
1 | 1

CONSTRAINT SYSTEM FROM ALGORITHM 1

                             Algorithm 1 Variables                              
--------------------------------------------------------------------------------
  ℵᴿ (aleph_R):      4  (number of response type combinations)
  B           :      4  (number of (W_L, W_R) configurations)
  dim(q)      :      4  (decision variable has 4 components)
  dim(p*)     :      4  (joint probability vector)
  |W_L configs|:      2  (number of distinct W_L configurations)
  dim(p|W_L)  :      2  (conditional prob. vector per W_L config)

                               Matrix Dimensions                                
--------------------------------------------------------------------------------
  P matri

In [49]:
print(constraints1.Lambda_matrix)

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
